In [9]:
import pandas as pd
import requests
import time
from tqdm import tqdm
from bs4 import BeautifulSoup

In [10]:
# legislatures = [57, 56, 55, 54, 53, 52]
legislatures = [56]

dfs = []

for legislature in legislatures:
    df = pd.read_csv(f'data/deputies/deputies_{legislature}.csv')
    dfs.append(df)

df = pd.concat(dfs)

In [11]:
already_extracted_deputies = pd.read_csv("data/speeches/speeches_56_1.csv")["deputy"].unique().tolist()
already_extracted_deputies_2 = pd.read_csv("data/speeches/speeches_56_2.csv")["deputy"].unique().tolist()
already_extracted_deputies_3 = pd.read_csv("data/speeches/speeches_56_3.csv")["deputy"].unique().tolist()
already_extracted_deputies = already_extracted_deputies + already_extracted_deputies_2 + already_extracted_deputies_3

df = df[~df["nome"].isin(already_extracted_deputies)]

In [12]:
print(f"Total de deputados para extrair: {df['nome'].nunique()}")

Total de deputados para extrair: 237


In [5]:
def extract_discourse_links(soup):
    """
    Function to extract the links to the discourses from the soup object.
    """
    # find all table rows
    table_rows = soup.find_all('td')
    
    links = []
    
    # iterate over the rows
    for row in table_rows:
        a_tags = row.find_all('a')
        if a_tags:
            for a_tag in a_tags:
                try:
                    link = a_tag['href']
                except:
                    continue
                    
                if link.startswith('TextoHTML'):
                    # remove "\n", "\r", and "\t"
                    link = link.replace('\n', '')
                    link = link.replace('\r', '')
                    link = link.replace('\t', '')
                    links.append(link)

    return links

In [6]:
deputies_names = set(df["nome"])
data = pd.DataFrame(columns=["deputy", "discourse", "discourse_link", "date", "phase", "expected_number_of_discourses"])
deputy_number = 1

FILENAME = "data/speeches/speeches_56_3.csv"

# iterate over the deputies
for deputy in deputies_names:
    current_page = 1
    all_discourse_links = []
    deputy_name = deputy.replace(" ", "+")

    # iterate over the pages of the deputy
    while True:
        try:
            # Get the page
            url_table = f"https://www.camara.leg.br/internet/sitaqweb/resultadoPesquisaDiscursos.asp?CurrentPage={current_page}&txOrador={deputy_name}&txPartido=&txUF=&dtInicio=&dtFim=&txTexto=&txSumario=&basePesq=plenario&CampoOrdenacao=dtSessao&PageSize=100&TipoOrdenacao=DESC&btnPesq=Pesquisar"
            response = requests.get(url_table)
        except Exception as e:
            print(f"Failed to get {deputy} - {e}")
            break
        
        # Check if the page was successfully retrieved
        if response.ok:
            soup = BeautifulSoup(response.text, 'html.parser')
            page_discourse_links = extract_discourse_links(soup)
        else:
            # If the page was not successfully retrieved, break the loop
            print(f"Failed to get {deputy} - {response} - page {current_page}")
            continue
        
        # If there are no more discourse links, break the loop
        if not page_discourse_links:
            break
        
        # Add the discourse links to the list
        print(f"Extracted {len(page_discourse_links)} discourse links for {deputy} - page {current_page}")
        all_discourse_links.extend(page_discourse_links)
        
        current_page += 1
        
        time.sleep(2)

    # Iterate over the discourse links
    for discourse in tqdm(all_discourse_links, desc=f"Processing Discourses of {deputy}", leave=False):
        # Get the discourse page
        try:
            url_discourse = f"https://www.camara.leg.br/internet/sitaqweb/{discourse}"
            response = requests.get(url_discourse)
        except Exception as e:
            print(f"Failed to get {url_discourse} of deputy {deputy} - {e}")
            continue
        
        # Check if the page was successfully retrieved
        if not response.ok:
            print(f"Failed to get {url_discourse} of deputy {deputy} - {response}")
            continue
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract discourse text
        discourse_text = soup.find_all('p')
        discourse_text = " ".join([p.text for p in discourse_text])
        
        # Extract date and phase
        date, phase = None, None
        tds_right = soup.find_all('td', align='right')
        for td in tds_right:
            if "Data" in td.text:
                date = td.text.split(":")[1].strip()  # Split to get date part
            elif "Fase" in td.text:
                phase = td.text.split(":")[1].strip()  # Split to get phase part

        # Add to dataframe
        data = pd.concat([data, pd.DataFrame([[deputy, discourse_text, url_discourse, date, phase, len(all_discourse_links)]], 
                                             columns=["deputy", "discourse", "discourse_link", "date", "phase", "expected_number_of_discourses"])])
        
        # Save every 10 deputies
        if deputy_number % 10 == 0:
            data.to_csv(FILENAME, index=False)
            
        time.sleep(0.5)
    
    print(f"✅ {deputy_number} Finished {deputy}\n")
    deputy_number += 1
    
    time.sleep(20)

# Save final result
data.to_csv(FILENAME, index=False)

Extracted 100 discourse links for Aliel Machado - page 1
Extracted 100 discourse links for Aliel Machado - page 2
Extracted 100 discourse links for Aliel Machado - page 3
Extracted 100 discourse links for Aliel Machado - page 4
Extracted 100 discourse links for Aliel Machado - page 5
Extracted 100 discourse links for Aliel Machado - page 6
Extracted 48 discourse links for Aliel Machado - page 7


✅ 1 Finished Aliel Machado

Extracted 17 discourse links for Eli Corrêa Filho - page 1


✅ 2 Finished Eli Corrêa Filho

Extracted 100 discourse links for Giovani Cherini - page 1
Extracted 100 discourse links for Giovani Cherini - page 2
Extracted 100 discourse links for Giovani Cherini - page 3
Extracted 100 discourse links for Giovani Cherini - page 4
Extracted 100 discourse links for Giovani Cherini - page 5
Extracted 100 discourse links for Giovani Cherini - page 6
Extracted 100 discourse links for Giovani Cherini - page 7
Extracted 100 discourse links for Giovani Cherini - page 8
Extracted 100 discourse links for Giovani Cherini - page 9
Extracted 100 discourse links for Giovani Cherini - page 10
Extracted 100 discourse links for Giovani Cherini - page 11
Extracted 9 discourse links for Giovani Cherini - page 12


✅ 3 Finished Giovani Cherini

Extracted 33 discourse links for Franco Cartafina - page 1


✅ 4 Finished Franco Cartafina

Extracted 100 discourse links for Bira do Pindaré - page 1
Extracted 100 discourse links for Bira do Pindaré - page 2
Extracted 100 discourse links for Bira do Pindaré - page 3
Extracted 100 discourse links for Bira do Pindaré - page 4
Extracted 100 discourse links for Bira do Pindaré - page 5
Extracted 100 discourse links for Bira do Pindaré - page 6
Extracted 100 discourse links for Bira do Pindaré - page 7
Extracted 3 discourse links for Bira do Pindaré - page 8


...

✅ 66 Finished Julio Lopes

Extracted 38 discourse links for Dimas Fabiano - page 1


✅ 67 Finished Dimas Fabiano

Extracted 100 discourse links for Sandro Alex - page 1
Extracted 100 discourse links for Sandro Alex - page 2
Extracted 100 discourse links for Sandro Alex - page 3
Extracted 100 discourse links for Sandro Alex - page 4
Extracted 100 discourse links for Sandro Alex - page 5
Extracted 100 discourse links for Sandro Alex - page 6
Extracted 48 discourse links for Sandro Alex - page 7


✅ 68 Finished Sandro Alex

Extracted 77 discourse links for Alfredinho - page 1


✅ 69 Finished Alfredinho

Extracted 100 discourse links for Gilson Marques - page 1
Extracted 100 discourse links for Gilson Marques - page 2
Extracted 100 discourse links for Gilson Marques - page 3
Extracted 100 discourse links for Gilson Marques - page 4
Extracted 100 discourse links for Gilson Marques - page 5
Extracted 100 discourse links for Gilson Marques - page 6
Extracted 98 discourse links for Gilson Marques - page 7


KeyboardInterrupt: 

In [8]:
data.to_csv("data/speeches/speeches_56_3.csv", index=False)